Author: Zaid Bin Tariq

Date: 2/27/2022

This notebook converts the raw data into training and testing set to be used for the neural network 

Data output might vary depending upon how the activity recognition problem is seen.

Possibily another notebook in future might be made for different types of data splits for the human activity recognition problem

In [1]:
import pickle,os
import numpy as np

In [14]:
%load_ext autoreload
%autoreload 2
import utils as ut

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First we will load the data was combined into a dictionary/lists for easy access. The following code is using the realDisp data and assumes that it was already split into a dictionary earlier

In [2]:
path_to_data = ''# Data path of the created dictionary
dictionary_filename = "All_data.pkl"
pickle_out = open(os.path.join(path_to_data,dictionary_filename),"rb")
dictionary = pickle.load(pickle_out) # dictionary contains the data
pickle_out.close()

In [3]:
total_subjects = len(dictionary) 
total_Activities = 33
num_source_activities =  int(np.round(0.8*total_Activities)) # number of activities used for training source network
num_target_activities = total_Activities - num_source_activities # number of activities used for traget network

In [63]:
total_Activities

33

In [5]:
## Flip the comments in the box incase source_activities,pkl does not exist
# src_act_array = np.random.choice(np.arange(total_Activities),size = num_source_activities,replace = False)
# with open('source_activities.pkl', 'wb') as handle:
#     pickle.dump(src_act_array, handle)
pickle_out = open('source_activities.pkl',"rb")
src_act_array = pickle.load(pickle_out)

trg_act_array = np.arange(total_Activities)[~np.isin(np.arange(total_Activities),src_act_array)]

In [100]:
filename = 'All_activity_instances_data.h5'

In [62]:
temp = ut.get_complete_data2(dictionary,np.arange(0,total_Activities),path_to_data,'All_activity_instances_data.h5',50,time_step = 250)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
3 32
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
4 25
4 26
4 27
4 28
4 29
4 30
4 31
4 32
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 24
5 25
5 26
5 27
5 28
5 29
5 30
5 31
5 32
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15


In [68]:
with open('labels_subjects.pkl', 'wb') as handle:
    pickle.dump(temp, handle)

In [67]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(temp[0],temp[1], test_size=0.2,random_state=25)

In [111]:
labels = np.array(temp[0])

In [112]:
from collections import Counter
from operator import itemgetter

ob = Counter(labels)
values = ob.values()
values
check = []
for val in values:
    check.append(val)

check = np.array(check)
class_total_samples = int(np.floor(np.mean(check[1:])))


In [98]:
from sklearn.utils import resample
spam_upsample = resample(temp_X,
             replace=True,
             n_samples=7,
             random_state=42)

In [143]:
import h5py
class_total_samples = 400
data_file = h5py.File(os.path.join(path_to_data,filename), 'r') 
X_data = data_file['X']

In [123]:
X_data[np.where(labels == 2)[0],:,:].shape

(850, 250, 117)

In [160]:
total_samples = class_total_samples *src_act_array.shape[0]
indices_array = np.zeros(total_samples)
new_labels = np.zeros(total_samples)
start = 0
end = start+ class_total_samples


for i in range(src_act_array.shape[0]):
    temp = np.where(labels == src_act_array[i])[0]
    if(temp.shape[0]<class_total_samples):
        replace_bool = True
    else:
        replace_bool = False
    indices_resampled = resample(temp,replace=replace_bool,n_samples=class_total_samples,random_state=42)
    indices_array[start:end] = indices_resampled
    new_labels[start:end] = src_act_array[i] 
    start = end
    end = start+class_total_samples


In [162]:
from sklearn.model_selection import train_test_split
indices_train, indices_Val, label_train, label_Val = train_test_split(indices_array,new_labels, test_size=0.2,random_state=25,shuffle = True)

In [234]:
X_train = np.zeros((indices_train.shape[0],X_data.shape[1],X_data.shape[2]))
X_Val = np.zeros((indices_Val.shape[0],X_data.shape[1],X_data.shape[2]))

for i in range(indices_train.shape[0]):
    X_train[i,:,:] = X_data[indices_train[i],:,:]
for i in range(indices_Val.shape[0]):
    X_Val[i,:,:] = X_data[indices_Val[i],:,:]

In [226]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
def convert_one_hot(arr):

    values = arr
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded
    # invert first example
#     inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])


In [179]:
# onehot_encoded[2,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [243]:
# X_train = np.random.random((10,250,117))
# y_train = convert_one_hot(np.arange(0,10))
# X_Val = np.random.random((10,250,117))
# y_Val = convert_one_hot(np.arange(0,10))


y_train = convert_one_hot(label_train)
y_Val = convert_one_hot(label_Val)

In [244]:
y_Val.shape

(2080, 26)

In [236]:
# Model ML
modelPath = "./models/"
modelName = "RNN_a1"

In [245]:
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
import tensorflow as tf
import json, os, argparse, imp

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = True  # to log device placement (on which device the operation ran)
# sess = tf.Session(config=config)
# set_session(sess)  # set this TensorFlow session as the default session for Keras
# model_module = imp.load_source('model_module', os.path.join(modelPath, modelName+'.py'))
model = model_check((250,117))
model.summary()



print('(+) Saving model: '+modelName)
json_string = model.to_json()
open(os.path.join(modelPath, modelName+'.json'), 'w').write(json_string)
ad =optimizers.Adam(lr = 0.01)
model.compile(loss='binary_crossentropy', optimizer=ad, metrics=['acc'])
# model.load_weights(os.path.join(modelPath, modelName+substr+'.wgt'))
# wait = input("Done")
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath=os.path.join(modelPath,modelName+'1.wgt'), 
                                verbose=1, 
                                save_best_only=True,monitor = 'val_acc')

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 50)                33600     
_________________________________________________________________
dense_19 (Dense)             (None, 26)                1326      
_________________________________________________________________
activation_15 (Activation)   (None, 26)                0         
Total params: 34,926
Trainable params: 34,926
Non-trainable params: 0
_________________________________________________________________
(+) Saving model: RNN_a1


In [246]:
fitlog = model.fit(X_train,y_train,batch_size = 256,epochs=10, verbose=1, 
                    callbacks=[checkpointer],
                    shuffle = "batch",validation_data = (X_Val,y_Val))
model.save_weights(filepath=os.path.join(modelPath,modelName+'1lastepoch.wgt'), overwrite=True)

Train on 8320 samples, validate on 2080 samples
Epoch 1/10
8320/8320 [==============================] - 9s 1ms/step - loss: 0.0811 - acc: 0.9735 - val_loss: 0.0303 - val_acc: 0.9903

Epoch 00001: val_acc improved from -inf to 0.99033, saving model to ./models/RNN_a11.wgt
Epoch 2/10
8320/8320 [==============================] - 8s 964us/step - loss: 0.0171 - acc: 0.9953 - val_loss: 0.0112 - val_acc: 0.9977

Epoch 00002: val_acc improved from 0.99033 to 0.99769, saving model to ./models/RNN_a11.wgt
Epoch 3/10
8320/8320 [==============================] - 8s 961us/step - loss: 0.0094 - acc: 0.9978 - val_loss: 0.0076 - val_acc: 0.9982

Epoch 00003: val_acc improved from 0.99769 to 0.99815, saving model to ./models/RNN_a11.wgt
Epoch 4/10
8320/8320 [==============================] - 8s 949us/step - loss: 0.0067 - acc: 0.9985 - val_loss: 0.0060 - val_acc: 0.9987

Epoch 00004: val_acc improved from 0.99815 to 0.99865, saving model to ./models/RNN_a11.wgt
Epoch 5/10
8320/8320 [===================

In [239]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape

def model_check(input_shape):
#     inp = Input(input_shape)
    model = Sequential()
    model.add(LSTM(50,input_shape = input_shape))
    model.add(Dense(26))
    model.add(Activation('softmax'))
    
#     encoded_l = model(inp)
    
    return model
# model.summary()